In [1]:
# Importing Libraries 
import pandas as pd
import numpy as np
import math
from pyomo.environ import *

ModuleNotFoundError: No module named 'pyomo'

In [3]:
# Importing pyomo Libraries
from __future__ import division
from pyomo.opt import SolverFactory

ModuleNotFoundError: No module named 'pyomo'

In [3]:
# Reading the data from Excel workbook
MgDemandDist=pd.read_excel("Staffing+Data.xlsx", sheet_name='DemandData')
StaffAv=pd.read_excel("Staffing+Data.xlsx",sheet_name='StaffAvailability')
Cost=pd.read_excel("Staffing+Data.xlsx",sheet_name='Cost')
transCost=pd.read_excel("Staffing+Data.xlsx",sheet_name='ServiceRate')

In [4]:
#To display Demand data set
MgDemandDist.head()

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942
3,A,Apr,2297
4,A,May,1992


In [12]:
#To display Staff average data set
StaffAv.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [13]:
#To display Cost data set
Cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180
3,A,Apr,60000,5000.0,180
4,A,May,60000,5000.0,180


In [14]:
#To display Service rate of application
transCost.head()

,MgAppServedPerMonth
0,40


## Data pre-processing 

In [15]:
# Create the required Python data structures for indexes and parameters

In [16]:
# Creating Index for States/Bank Locations
BankLoc=Cost['State'].unique()

#Creating Index for  Months
Month=Cost['Month'].unique()

In [17]:
#Defining parameter for Demand
Demand=MgDemandDist.set_index(['State','Month'])['Demand'].to_dict()

#Defining prameter for FTE Salary
FTE_Salary=Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

#Defining parameter for Unit outsource cost
UnitOutSourceCost=Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Number of Application served with  100% availability
FTE_AppServeRate=transCost.iloc[0,0]

#Defining parameter for Minimum Staff availability
StaffAv_LB=StaffAv.set_index(['State','Month'])['LB'].to_dict()

#Defining parameter for Maximum Staff availability
StaffAv_UB=StaffAv.set_index(['State','Month'])['UB'].to_dict()

#Defining parameter for Average Staff availability
StaffAvPer=StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()



# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



In [18]:
#Staffing Recommendations: Answered with Mathematical Expression have been mentioned in report. 


**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [19]:
# Creating a model instance
model=ConcreteModel()

NameError: name 'ConcreteModel' is not defined

In [557]:
# Define Pyomo sets and Parameters

#Defining Set for State and Months
model.i=Set(initialize=BankLoc.tolist(),doc="State")
model.j=Set(initialize=Month.tolist(),doc="Months")

#Defining parameter for Demand
Demand=MgDemandDist.set_index(['State','Month'])['Demand'].to_dict()

#Defining prameter for FTE Salary
FTE_Salary=Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

#Defining parameter for Unit outsource cost
UnitOutSourceCost=Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Number of Application served with  100% availability
FTE_AppServeRate=transCost.iloc[0,0]

#Defining parameter for Minimum Staff availability
StaffAv_LB=StaffAv.set_index(['State','Month'])['LB'].to_dict()

#Defining parameter for Maximum Staff availability
StaffAv_UB=StaffAv.set_index(['State','Month'])['UB'].to_dict()

#Defining parameter for Average Staff availability
StaffAvPer=StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()

model.d=Param(model.i,model.j,initialize=Demand,doc='Demand')
model.s=Param(model.i,model.j,initialize=FTE_Salary,doc='FTE_Salary')
model.oc=Param(model.i,model.j,initialize=UnitOutSourceCost,doc='UnitOutSourceCost')
model.sa=Param(model.i,model.j,initialize=StaffAvPer,doc='StaffAvPer')

#Scalar Application Service rate when 100% working
model.r=Param(initialize=FTE_AppServeRate,doc='FTE_AppServeRate')

#Paramter for no of application that can be processed for given staff availability
def c_FTEAPP(model,i,j):
  return model.r* model.sa[i,j]

model.FTEAPPNO=Param(model.i,model.j,initialize=c_FTEAPP,doc='No of FTE APP')

# Prameter for Regulatory restriction for STate A
def Reg_A(model,i,j):
  return 0.30*model.d[i,j]

model.Reg_A=Param(model.i,model.j,initialize=Reg_A,doc='RegRest_A')

# Prameter for Regulatory restriction for STate B
def Reg_B(model,i,j):
  return 0.40*model.d[i,j]

model.Reg_B=Param(model.i,model.j,initialize=Reg_B,doc='RegRest_B')

In [558]:
# Decision variables

model.x= Var(model.i,model.j,domain=NonNegativeReals,doc='No of FTE')
model.y= Var(model.i,model.j,domain=NonNegativeIntegers,doc='No of Outsorced Apps')

In [559]:
# Constraints

# Demand Constraint
model.demand_constraint=ConstraintList()
for i in model.i:
  for j in model.j:
    model.demand_constraint.add(expr=model.x[i,j]*model.FTEAPPNO[i,j] + model.y[i,j]==model.d[i,j])


#Regulatory constraint
model.regulatory_constraint=ConstraintList()
for i in model.i:
  for j in model.j:
    if i =='A':
      model.regulatory_constraint.add(expr= model.y['A',j] <=model.Reg_A['A',j])
    elif i=='B':
      model.regulatory_constraint.add(expr= model.y['B',j] <=model.Reg_B['B',j])


In [560]:
# Objective function

# To Mininmize cost
def objective_rule(model):
  return sum(model.x[i,j]*model.s[i,j] for i in model.i for j in model.j) + sum(model.y[i,j]*model.oc[i,j] for i in  model.i for j in model.j)

model.objective=Objective(rule=objective_rule, sense=minimize,doc='Define Objective Function')

In [561]:
# Invoking the solver

result_cost=SolverFactory('glpk').solve(model)
result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.013334035873413086
# ----------------------------------------------------------
#   Solution Information
# -----------

In [562]:
# Print the value of the objective function

TotalCost=model.objective.expr()
TotalCost


17962336.448769882

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [563]:
#Total no of Outsorced applications
s=0
for i in model.i:
  for j in model.j:
    s=s+model.y[i,j].value
print(s)

20749.0


In [564]:
#Percentage of Outsorced applications
TotalApplDemand=MgDemandDist['Demand'].sum()
PercentageOutSourced=(s/TotalApplDemand)*100
print(PercentageOutSourced)

18.314297314950483


In [565]:
# To display the Model fileds and Result
model.pprint()

13 Set Declarations
    FTEAPPNO_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C', 'Sep'), ('C', 'Oct'), ('C', 'Nov'), ('C', 'Dec')}
    Reg_A_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), (

In [566]:
#To display Model values along with Constraints and Objective Function
model.display()

Model unknown

  Variables:
    x : No of FTE
        Size=36, Index=x_index
        Key          : Lower : Value            : Upper : Fixed : Stale : Domain
        ('A', 'Apr') :     0 :         71.78125 :  None : False : False : NonNegativeReals
        ('A', 'Aug') :     0 : 110.888157894737 :  None : False : False : NonNegativeReals
        ('A', 'Dec') :     0 : 139.576923076923 :  None : False : False : NonNegativeReals
        ('A', 'Feb') :     0 : 160.460526315789 :  None : False : False : NonNegativeReals
        ('A', 'Jan') :     0 : 161.728395061728 :  None : False : False : NonNegativeReals
        ('A', 'Jul') :     0 : 137.279411764706 :  None : False : False : NonNegativeReals
        ('A', 'Jun') :     0 : 77.9109589041096 :  None : False : False : NonNegativeReals
        ('A', 'Mar') :     0 : 198.066666666667 :  None : False : False : NonNegativeReals
        ('A', 'May') :     0 : 63.8461538461538 :  None : False : False : NonNegativeReals
        ('A', 'Nov') : 

In [567]:
#Calculating the Result in Column format
output=[]

for i in model.i:
  for j in model.j:
    No_Of_FTE=model.x[i,j].value
    No_of_OutsorcedApps=model.y[i,j].value
    Demand=model.d[i,j]
    StaffAvPer=model.sa[i,j]
    output.append([i,j,Demand,StaffAvPer,No_Of_FTE,No_of_OutsorcedApps,Demand])

print(output)

[['A', 'Jan', 5240, 0.81, 161.728395061728, 0.0, 5240], ['A', 'Feb', 4878, 0.76, 160.460526315789, 0.0, 4878], ['A', 'Mar', 5942, 0.75, 198.066666666667, 0.0, 5942], ['A', 'Apr', 2297, 0.8, 71.78125, 0.0, 2297], ['A', 'May', 1992, 0.78, 63.8461538461538, 0.0, 1992], ['A', 'Jun', 2275, 0.73, 77.9109589041096, 0.0, 2275], ['A', 'Jul', 5334, 0.68, 137.279411764706, 1600.0, 5334], ['A', 'Aug', 3371, 0.76, 110.888157894737, 0.0, 3371], ['A', 'Sep', 3759, 0.81, 116.018518518519, 0.0, 3759], ['A', 'Oct', 3529, 0.73, 120.856164383562, 0.0, 3529], ['A', 'Nov', 4284, 0.68, 110.257352941176, 1285.0, 4284], ['A', 'Dec', 5183, 0.65, 139.576923076923, 1554.0, 5183], ['B', 'Jan', 4927, 0.81, 152.067901234568, 0.0, 4927], ['B', 'Feb', 2628, 0.76, 51.875, 1051.0, 2628], ['B', 'Mar', 2974, 0.75, 59.5, 1189.0, 2974], ['B', 'Apr', 2338, 0.8, 73.0625, 0.0, 2338], ['B', 'May', 4020, 0.78, 128.846153846154, 0.0, 4020], ['B', 'Jun', 3147, 0.73, 64.6917808219178, 1258.0, 3147], ['B', 'Jul', 4271, 0.68, 94.2279

In [568]:
# Creating dataframe for the results
df_result=pd.DataFrame(output,columns=['State','Month','Demand','StaffAvPer','FTE','Apps_Outsorced','FTE_APP_Served'])
df_result


,State,Month,Demand,StaffAvPer,FTE,Apps_Outsorced,FTE_APP_Served
0,A,Jan,5240,0.81,161.728395,0.0,5240
1,A,Feb,4878,0.76,160.460526,0.0,4878
2,A,Mar,5942,0.75,198.066667,0.0,5942
3,A,Apr,2297,0.80,71.781250,0.0,2297
4,A,May,1992,0.78,63.846154,0.0,1992
5,A,Jun,2275,0.73,77.910959,0.0,2275
6,A,Jul,5334,0.68,137.279412,1600.0,5334
7,A,Aug,3371,0.76,110.888158,0.0,3371
8,A,Sep,3759,0.81,116.018519,0.0,3759
9,A,Oct,3529,0.73,120.856164,0.0,3529


In [569]:
# Writing the results in to an Excel sheet

df_result.to_excel('Result_2.xlsx',sheet_name='Result',index=False)


# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [570]:
# Creating a model instance
model=ConcreteModel()

In [571]:
# Creating the Indexes

#For Unique States
BankLoc=Cost['State'].unique()
#For Unique Months
Month=Cost['Month'].unique()


In [572]:
# Define Pyomo sets and Parameters

#Defining Set for State and Months
model.i=Set(initialize=BankLoc.tolist(),doc="State")
model.j=Set(initialize=Month.tolist(),doc="Months")

#Defining parameter for Demand
Demand=MgDemandDist.set_index(['State','Month'])['Demand'].to_dict()

#Defining prameter for FTE Salary
FTE_Salary=Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

#Defining parameter for Unit outsource cost
UnitOutSourceCost=Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Number of Application served with  100% availability
FTE_AppServeRate=transCost.iloc[0,0]

#Defining parameter for Minimum Staff availability
StaffAv_LB=StaffAv.set_index(['State','Month'])['LB'].to_dict()

#Defining parameter for Maximum Staff availability
StaffAv_UB=StaffAv.set_index(['State','Month'])['UB'].to_dict()

#Defining parameter for Average Staff availability
StaffAvPer=StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()

model.d=Param(model.i,model.j,initialize=Demand,doc='Demand')
model.s=Param(model.i,model.j,initialize=FTE_Salary,doc='FTE_Salary')
model.oc=Param(model.i,model.j,initialize=UnitOutSourceCost,doc='UnitOutSourceCost')
model.lb=Param(model.i,model.j,initialize=StaffAv_LB,doc='StaffAv_LB')

#Scalar Application Service rate when 100% working
model.r=Param(initialize=FTE_AppServeRate,doc='FTE_AppServeRate')


#Paramter for no of application that can be processed for given staff availability
def c_FTEAPP(model,i,j):
  return model.r* model.lb[i,j]

model.FTEAPPNO=Param(model.i,model.j,initialize=c_FTEAPP,doc='No of FTE APP')

# Prameter for Regulatory restriction for STate A
def Reg_A(model,i,j):
  return 0.30*model.d[i,j]

model.Reg_A=Param(model.i,model.j,initialize=Reg_A,doc='RegRest_A')

# Prameter for Regulatory restriction for STate B
def Reg_B(model,i,j):
  return 0.40*model.d[i,j]

model.Reg_B=Param(model.i,model.j,initialize=Reg_B,doc='RegRest_B')


In [573]:
# Decision variables

model.x= Var(model.i,model.j,domain=NonNegativeReals,doc='No of FTE')
model.y= Var(model.i,model.j,domain=NonNegativeReals,doc='No of Outsorced Apps')

In [574]:
# Constraints

# Demand Constraint
model.demand_constraint=ConstraintList()
for i in model.i:
  for j in model.j:
    model.demand_constraint.add(expr=model.x[i,j]*model.FTEAPPNO[i,j] + model.y[i,j]==model.d[i,j])

#Regulatory constraint
model.regulatory_constraint=ConstraintList()
for i in model.i:
  for j in model.j:
    if i =='A':
      model.regulatory_constraint.add(expr= model.y['A',j] <=model.Reg_A['A',j])
    elif i=='B':
      model.regulatory_constraint.add(expr= model.y['B',j] <=model.Reg_B['B',j])

In [575]:
# Objective function

#Mininmize cost
def objective_rule(model):
  return sum(model.x[i,j]*model.s[i,j] for i in model.i for j in model.j) + sum(model.y[i,j]*model.oc[i,j] for i in  model.i for j in model.j)

model.objective=Objective(rule=objective_rule, sense=minimize,doc='Define Objective Function')

In [576]:
# Invoking the solver

result_cost=SolverFactory('glpk').solve(model)
result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599283.3977411
  Upper bound: 19599283.3977411
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.019633769989013672
# ----------------------------------------------------------
#   Solution Information
# -----------

In [577]:
# Print the value of the objective function
TotalCost_LB=model.objective.expr()
TotalCost_LB

19599283.397741143

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [578]:
#Calculating the Result in Column format
output=[]

for i in model.i:
  for j in model.j:
    No_Of_FTE=model.x[i,j].value
    No_of_OutsorcedApps=model.y[i,j].value
    Demand=model.d[i,j]
    LB=model.lb[i,j]
    output.append([i,j,Demand,LB,No_Of_FTE,No_of_OutsorcedApps,Demand])


In [579]:
# Creating dataframe for the results
df_result=pd.DataFrame(output,columns=['State','Month','Demand','LB','FTE','Apps_Outsorced','FTE_APP_Served'])
df_result

,State,Month,Demand,LB,FTE,Apps_Outsorced,FTE_APP_Served
0,A,Jan,5240,0.70,187.142857,0.0,5240
1,A,Feb,4878,0.65,131.330769,1463.4,4878
2,A,Mar,5942,0.70,212.214286,0.0,5942
3,A,Apr,2297,0.75,76.566667,0.0,2297
4,A,May,1992,0.70,71.142857,0.0,1992
5,A,Jun,2275,0.65,61.250000,682.5,2275
6,A,Jul,5334,0.60,155.575000,1600.2,5334
7,A,Aug,3371,0.65,90.757692,1011.3,3371
8,A,Sep,3759,0.70,134.250000,0.0,3759
9,A,Oct,3529,0.65,95.011538,1058.7,3529


In [580]:
# Writing the results in to an Excel sheet

df_result.to_excel('Result_3.xlsx',sheet_name='Result',index=False)

#### 3.1.2 Percentage of outsourced applications 

In [581]:
#Total no of Outsorced applicants actual
noOutSource=0
for i in model.i:
  for j in model.j:
    noOutSource=noOutSource+model.y[i,j].value
print(noOutSource)


39816.600000000006


In [582]:
#Total no of applications Demand
TotalApplDemand=MgDemandDist['Demand'].sum()

#Percentage of Outsorced applictns
PercentageOutSourced=(s/TotalApplDemand)*100
print(PercentageOutSourced)

18.314297314950483


#### 3.1.3 Average cost per application

In [583]:
# write your code here

print('Averge cost per Application in Case of Minimum(LB) Staff availability is :',TotalCost_LB/TotalApplDemand)

Averge cost per Application in Case of Minimum(LB) Staff availability is : 172.9948929134918


### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [584]:
# Creating a model instance

model=ConcreteModel()

In [585]:
# Creating the Indexes
#For Unique ank locations
BankLoc=Cost['State'].unique()
#For Unique Months
Month=Cost['Month'].unique()

In [586]:
# Define Pyomo sets and Parameters

#Defining Set for State and Months
model.i=Set(initialize=BankLoc.tolist(),doc="State")
model.j=Set(initialize=Month.tolist(),doc="Months")

MgDemandDist=pd.read_excel("Staffing+Data.xlsx", sheet_name='DemandData')
StaffAv=pd.read_excel("Staffing+Data.xlsx",sheet_name='StaffAvailability')
Cost=pd.read_excel("Staffing+Data.xlsx",sheet_name='Cost')
transCost=pd.read_excel("Staffing+Data.xlsx",sheet_name='ServiceRate')


#Defining parameter for Demand
Demand=MgDemandDist.set_index(['State','Month'])['Demand'].to_dict()

#Defining prameter for FTE Salary
FTE_Salary=Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

#Defining parameter for Unit outsource cost
UnitOutSourceCost=Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Number of Application served with  100% availability
FTE_AppServeRate=transCost.iloc[0,0]

#Defining parameter for Minimum Staff availability
StaffAv_LB=StaffAv.set_index(['State','Month'])['LB'].to_dict()

#Defining parameter for Maximum Staff availability
StaffAv_UB=StaffAv.set_index(['State','Month'])['UB'].to_dict()

#Defining parameter for Average Staff availability
StaffAvPer=StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()

model.d=Param(model.i,model.j,initialize=Demand,doc='Demand')
model.s=Param(model.i,model.j,initialize=FTE_Salary,doc='FTE_Salary')
model.oc=Param(model.i,model.j,initialize=UnitOutSourceCost,doc='UnitOutSourceCost')
model.ub=Param(model.i,model.j,initialize=StaffAv_UB,doc='StaffAv_UB')

#Scalar Application Service rate when 100% working
model.r=Param(initialize=FTE_AppServeRate,doc='FTE_AppServeRate')

#Paramter for no of application that can be processed for given staff availability
def c_FTEAPP(model,i,j):
  return model.r* model.ub[i,j]

model.FTEAPPNO=Param(model.i,model.j,initialize=c_FTEAPP,doc='No of FTE APP')

# Prameter for Regulatory restriction for STate A
def Reg_A(model,i,j):
  return 0.30*model.d[i,j]

model.Reg_A=Param(model.i,model.j,initialize=Reg_A,doc='RegRest_A')

# Prameter for Regulatory restriction for STate B
def Reg_B(model,i,j):
  return 0.40*model.d[i,j]

model.Reg_B=Param(model.i,model.j,initialize=Reg_B,doc='RegRest_B')

In [587]:
# Decision variables
model.x= Var(model.i,model.j,domain=NonNegativeReals,doc='No of FTE')
model.y= Var(model.i,model.j,domain=NonNegativeReals,doc='No of Outsorced Apps')

In [588]:
# Constraints

# Demand Constraint
model.demand_constraint=ConstraintList()
for i in model.i:
  for j in model.j:
    model.demand_constraint.add(expr=model.x[i,j]*model.FTEAPPNO[i,j] + model.y[i,j]==model.d[i,j])



#Regulatory constraint
model.regulatory_constraint=ConstraintList()
for i in model.i:
  for j in model.j:
    if i =='A':
      model.regulatory_constraint.add(expr= model.y['A',j] <=model.Reg_A['A',j])
    elif i=='B':
      model.regulatory_constraint.add(expr= model.y['B',j] <=model.Reg_B['B',j])


In [589]:
# Objective function
# To Mininmize cost
def objective_rule(model):
  return sum(model.x[i,j]*model.s[i,j] for i in model.i for j in model.j) + sum(model.y[i,j]*model.oc[i,j] for i in  model.i for j in model.j)

model.objective=Objective(rule=objective_rule, sense=minimize,doc='Define Objective Function')

In [590]:
# Invoking the solver
result_cost=SolverFactory('glpk').solve(model)
result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527521.3522214
  Upper bound: 16527521.3522214
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.012503385543823242
# ----------------------------------------------------------
#   Solution Information
# -----------

In [591]:
# Print the value of the objective function
TotalCost_UB=model.objective.expr()
TotalCost_UB


16527521.352221444

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [592]:
#Calculating the Result in Column format
output=[]

for i in model.i:
  for j in model.j:
    No_Of_FTE=model.x[i,j].value
    No_of_OutsorcedApps=model.y[i,j].value
    Demand=model.d[i,j]
    UB=model.ub[i,j]
    output.append([i,j,Demand,UB,No_Of_FTE,No_of_OutsorcedApps,Demand])

In [593]:
# Creating dataframe for the results
df_result=pd.DataFrame(output,columns=['State','Month','Demand','UB','FTE','Apps_Outsorced','FTE_APP_Served'])
df_result


,State,Month,Demand,UB,FTE,Apps_Outsorced,FTE_APP_Served
0,A,Jan,5240,0.90,145.555556,0.0,5240
1,A,Feb,4878,0.85,143.470588,0.0,4878
2,A,Mar,5942,0.80,185.687500,0.0,5942
3,A,Apr,2297,0.85,67.558824,0.0,2297
4,A,May,1992,0.85,58.588235,0.0,1992
5,A,Jun,2275,0.80,71.093750,0.0,2275
6,A,Jul,5334,0.75,177.800000,0.0,5334
7,A,Aug,3371,0.85,99.147059,0.0,3371
8,A,Sep,3759,0.90,104.416667,0.0,3759
9,A,Oct,3529,0.80,110.281250,0.0,3529


In [594]:
# Writing the results in to an Excel sheet

df_result.to_excel('Result_4.xlsx',sheet_name='Result',index=False)

#### 3.2.2 Percentage of outsourced applications

In [595]:
#Total Outsorced applications- Actual
s=0
for i in model.i:
  for j in model.j:
    s=s+model.y[i,j].value
print(s)


4654.0


In [596]:
#Total no of applications demand
TotalApplDemand=MgDemandDist['Demand'].sum()

#Percentage of Outsorced applications
PercentageOutSourced=(s/TotalApplDemand)*100
print(PercentageOutSourced)

4.1078962698819


#### 3.2.3 Average cost per application

In [597]:
print('Averge cost per Application in Case of Maximum(UB) Staff availability is :',TotalCost_UB/TotalApplDemand)

Averge cost per Application in Case of Maximum(UB) Staff availability is : 145.88170028617088


# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

In [598]:
#Answered with Visualization have been mentioned in Report